In [147]:
import os
import pandas as pd
from linux_xbbg import blp

bt_dir = '../adr_results/turnover_lookback/'
adr_info = pd.read_csv('../data/raw/adr_info.csv')
adr_info['adr_ticker'] = adr_info['adr'].str.replace(' US Equity','')
trade_df = pd.read_csv(os.path.join(bt_dir, 'trades.csv'), parse_dates=['timestamp'])
trade_df['date'] = trade_df['timestamp'].dt.strftime('%Y-%m-%d')

In [148]:
p = trade_df.sort_values('timestamp').groupby(['date','ticker'])[['size','price']].first().loc['2025-12-16']

In [149]:
p['ISIN'] = blp.bds((p.index + ' US Equity').to_list(),'ID_ISIN')['value'].values

In [150]:
betas = pd.read_csv('../data/processed/models/ordinary_betas_index_only.csv', index_col=0).loc['2025-12-16'].rename('beta')

In [143]:
merged = p.merge(betas, left_index=True, right_index=True)
merged = merged.merge(adr_info[['adr_ticker','index_future_bbg','currency']], left_index=True, right_on='adr_ticker')
merged.loc[merged['index_future_bbg']=='Z', 'index_future_bbg'] = 'Z '
merged['index_future_bbg'] += 'H6 Index'
merged['val'] = merged['size'] * merged['price']

fut_price = blp.bdh(merged['index_future_bbg'].unique().tolist(), ['PX_LAST'], '2025-12-16', '2025-12-16').droplevel(1, axis=1).iloc[0]
multiplier = blp.bds(merged['index_future_bbg'].unique().tolist(), ['PRICE_MULTIPLIER']).rename(columns={'value':'PRICE_MULTIPLIER'})
currency = blp.bds(merged['index_future_bbg'].unique().tolist(), ['CURRENCY'])
currency['fx_rate'] = blp.bdh((currency['value'] + 'USD Curncy').tolist(), ['PX_LAST'], '2025-12-16', '2025-12-16').droplevel(1, axis=1).iloc[0].values
fut_merged = fut_price.to_frame('fut_price').merge(multiplier, left_index=True, right_index=True).merge(currency[['fx_rate']], left_index=True, right_index=True)
fut_merged['fut_notional_usd'] = fut_merged['fut_price'] * fut_merged['PRICE_MULTIPLIER'] * fut_merged['fx_rate']

merged = merged.merge(fut_merged[['fut_notional_usd']], left_on='index_future_bbg', right_index=True)
merged['size'] = -(merged['val'] * merged['beta'] / merged['fut_notional_usd'])
futures_pos = merged.groupby('index_future_bbg')[['size']].sum().round()
futures_isin = blp.bds(futures_pos.index.tolist(),'ID_ISIN').rename(columns={'value':'ISIN'})
futures_pos = futures_pos.merge(futures_isin, left_index=True, right_index=True, how='left')
futures_pos = futures_pos.reset_index(names='bloomberg_symbol')[['bloomberg_symbol','ISIN','size']]

In [ ]:
equity_portfolio = p.reset_index().rename(columns={'ticker':'bloomberg_symbol'})[['bloomberg_symbol','ISIN','size']]
equity_portfolio['bloomberg_symbol'] += ' US Equity'
equity_future_portfolio = equity_portfolio[~equity_portfolio['bloomberg_symbol'].isin(['EWU US Equity',
                                                                                       'EWJ US Equity',
                                                                                       'EWA US Equity',
                                                                                        'FEZ US Equity',])]
equity_future_portfolio = pd.concat([equity_future_portfolio,
                                    futures_pos], ignore_index=True)
equity_future_portfolio['size'] = equity_future_portfolio['size'].astype(int)

In [162]:
equity_portfolio.to_excel(os.path.join('..','equities_only_portfolio.xlsx'), index=False)
equity_future_portfolio.to_excel(os.path.join('..','futures_and_equities_portfolio.xlsx'), index=False)